![title](https://www.sportsgamblingpodcast.com/wp-content/uploads/2018/05/russia-2018.jpeg)

# **Update your group fixture results & table for the FIFA World Cup 2018**

This notebook allows you to **update & reset your Group Fixtures Results** and **populate the Group Tables for the World Cup**.

In [1]:
import pandas as pd
import psycopg2

## **How to use this notebook**

As a team:
- Follow the instructions below each heading.
- Only change code where asked to.
- See if you can improve the data code.

### **1. Specifiy your TEAM's Database details**

_Enter your Database details here:_

In [2]:
# specify your database details

database = # TEAM Database Name
user = # TEAM user
password = # TEAM password
host = # TEAM Amazon RDS End Point
port = # TEAM port number

# connect using psycopg2
conn = psycopg2.connect(database=database, user=user, password=password, host=host, port=port)
cursor = conn.cursor()

### **2. HELPER FUNCTIONS**

_run code to load all helper functions_

In [3]:
# get the tables

sql_teams = """SELECT * from teams"""
sql_group_fixtures = """SELECT * from group_fixtures"""
sql_group_team_results = """SELECT * from group_team_results"""
sql_group_teams = """SELECT * from group_teams"""

In [4]:
# helper functions function will fecth all the necessary tables and build a master table in order for us to easily view the fixtures
def get_master_table(sql_teams, sql_group_fixtures, sql_group_team_results, sql_group_teams):
    """ This function merges all the necessary tables from our database in order to identify which teams are linked to which fixture id
    """
    df_teams = pd.read_sql(sql_teams, conn).sort_values(by='id')
    df_group_fixtures = pd.read_sql(sql_group_fixtures, conn).sort_values(by='id')
    df_group_team_results = pd.read_sql(sql_group_team_results, conn).sort_values(by='id')
    df_group_teams = pd.read_sql(sql_group_teams, conn).sort_values(by='id')

    df_master = pd.merge(df_group_team_results, df_group_fixtures, how='left', left_on=['fixture_id'], right_on=['id']).merge(df_teams, how='left', left_on=['team_id'], right_on=['id'])
    df_master['date_filter'] = df_master['date'].apply(lambda x: pd.to_datetime(x))
    
    return df_master

def get_name_to_id_dict(sql_teams):
    """ returns a dictionary that can be used to link a Team Name to a Team ID
    """
    dict_name_to_id = pd.read_sql(sql_teams, conn).sort_values(by='id').set_index('long_name')[['id']].to_dict(orient='dict')['id']
    return dict_name_to_id

def get_teamid_to_groupid_dict(sql_group_teams):
    """ returns a dictionary that can be used to link a Team ID to a Group
    """
    dict_teamid_to_groupid = pd.read_sql(sql_group_teams, conn).sort_values(by='id').set_index('team_id')[['group_id']].to_dict(orient='dict')['group_id']
    return dict_teamid_to_groupid

def calc_team_points(team1, team1_goals, team2, team2_goals):
    """ function used to calculate the points for each team, based on goals scored
    """
    if team1_goals > team2_goals:
        team1_points = 3
        team2_points = 0
    elif team1_goals < team2_goals:
        team1_points = 0
        team2_points = 3
    else:
        team1_points = 1
        team2_points = 1
    return team1_points, team2_points    

def sql_update_execute(sql_update_team):
    """ run a SQL query in order to update the database
    """
    try:
        cursor.execute(sql_update_team)
        conn.commit()
    except: 
        conn.rollback()

def update_group_team_results(fixture_id, team1_longname, team2_longname, team1_goals, team2_goals):
    """ function is used to updated the Group Team Results table
    """
    fixture_id = fixture_id
    team1 = team1_longname
    team2 = team2_longname
    team1_goals = team1_goals
    team2_goals = team2_goals
    
    team1_id = get_name_to_id_dict(sql_teams)[team1]
    team2_id = get_name_to_id_dict(sql_teams)[team2]
    
    team1_points, team2_points = calc_team_points(team1, team1_goals, team2, team2_goals)

    sql_update_team1 = """ UPDATE group_team_results SET goals_for={:d}, goals_against={:d}, points={:d} WHERE fixture_id={:d} AND team_id={:d}""".format(team1_goals, team2_goals, team1_points, fixture_id, team1_id)
    sql_update_team2 = """ UPDATE group_team_results SET goals_for={:d}, goals_against={:d}, points={:d} WHERE fixture_id={:d} AND team_id={:d}""".format(team2_goals, team1_goals, team2_points, fixture_id, team2_id)
    
    sql_update_execute(sql_update_team1)
    sql_update_execute(sql_update_team2)
    
    return print('Match results updated')

# the following function will determine if a fixture was played or not by looking at the aggregate points for the match accross the 2 teams
def played_indicator(row):
    """ determine whether the game has been played or not, group by fixture_id first
    """
    if row['points'] > 0:
        pld_indi = 1
    else:
        pld_indi=0
    return pld_indi


# the following functions will create a Win | Draw | Loss indicator so that we can populate the Group Table stats
def win_indicator(row):
    if (row['points'] == 3) & (row['pld_indi'] == 1):
        w = 1
    else:
        w = 0
    return w

def draw_indicator(row):
    if (row['points'] == 1) & (row['pld_indi'] == 1):
        d = 1
    else:
        d = 0
    return d

def loose_indicator(row):
    if (row['points'] == 0) & (row['pld_indi'] == 1):
        l = 1
    else:
        l = 0
    return l   

# this function is used to execute our fixtures reset
def sql_reset_execute(fix_id):
    sql_reset_fixture = """UPDATE group_team_results SET points=0, goals_for=0, goals_against=0 WHERE fixture_id = {:d}""".format(int(fix_id))
    try:
        cursor.execute(sql_reset_fixture)
        conn.commit()
    except: 
        conn.rollback()

In [5]:
#map team id's to groups using a dictionary
dict_teamid_group = get_teamid_to_groupid_dict(sql_group_teams)

## **4. Create Master Fixture Table**

_Navigate this Pandas Dataframe to decide which fixture results you want to update / change_

In [6]:
df_master = get_master_table(sql_teams, sql_group_fixtures, sql_group_team_results, sql_group_teams)

_you can FILTER the table by date_

In [7]:
date = '2018-06-15'  #use Format: YYYY-MM-DD

In [8]:
df_date_filter = df_master[df_master['date_filter'] == date].loc[:,['date', 'fixture_id', 'long_name']]

In [9]:
df_date_filter

,date,fixture_id,long_name
2,2018-06-15,2,Uruguay
3,2018-06-15,2,Egypt
4,2018-06-15,3,Morocco
5,2018-06-15,3,IR Iran
6,2018-06-15,4,Portugal
7,2018-06-15,4,Spain


## **5. Change the Fixture Results ** 

_Enter the match details here that you would like to change / update_

In [10]:
### ENTER FIXTURE RESULTS HERE ###

# format            [ fixture_id, team1_name, team1_goals, team2_name, team_2_goals]
fixture_results =[
                   [1, 'Russia', 5, 'Saudi Arabia', 0],
                   [2, 'Uruguay', 1, 'Egypt', 0],
                   [3, 'Morocco' , 0, 'IR Iran', 1],
                   [4, 'Portugal', 3, 'Spain', 3],
                   [5, 'Australia', 1, 'France', 2],
                   [6, 'Argentina', 1, 'Iceland', 1],
                   [7, 'Peru', 0, 'Denmark', 1],
                   [8, 'Croatia', 2, 'Nigeria', 0],
                   [9, 'Costa Rica', 0, 'Serbia', 1],
                   [10, 'Mexico', 1, 'Germany', 0],
                   [11, 'Switzerland', 1, 'Brazil', 1]
   
]

Note that if you have previously updated fixture results, you do not have to run it again!

## **3. Update individual fixtures**

_RUN code below to update the fixture results_

In [11]:
for i in range(len(fixture_results)):
    fixture_id = fixture_results[i][0]
    team1_name = fixture_results[i][1]
    team2_name = fixture_results[i][3]
    team1_goals = fixture_results[i][2] 
    team2_goals = fixture_results[i][4]
    
    if (team1_goals == '') | (team2_goals == ''):
        print('please enter goals for fixture id: {:d}'.format(fixture_results[i][0]))
    else:
        update_group_team_results(fixture_id, team1_name, team2_name, team1_goals, team2_goals)
        print('Fixture id {:d} {:s} vs {:s} has been updated'.format(fixture_results[i][0], fixture_results[i][1], fixture_results[i][3]))

Match results updated
Fixture id 1 Russia vs Saudi Arabia has been updated
Match results updated
Fixture id 2 Uruguay vs Egypt has been updated
Match results updated
Fixture id 3 Morocco vs IR Iran has been updated
Match results updated
Fixture id 4 Portugal vs Spain has been updated
Match results updated
Fixture id 5 Australia vs France has been updated
Match results updated
Fixture id 6 Argentina vs Iceland has been updated
Match results updated
Fixture id 7 Peru vs Denmark has been updated
Match results updated
Fixture id 8 Croatia vs Nigeria has been updated
Match results updated
Fixture id 9 Costa Rica vs Serbia has been updated
Match results updated
Fixture id 10 Mexico vs Germany has been updated
Match results updated
Fixture id 11 Switzerland vs Brazil has been updated


## **4. Reset fixutres**

_Enter the fixtures that you would like to reset, as if the game was never played_

In [12]:
### ENTER FIXTURE ID's HERE ###

list_reset_fixtures = []

_Run code below to reset fixtures stats_

In [13]:
for i in range(len(list_reset_fixtures)):
    
    sql_reset_execute(list_reset_fixtures[i])
    print('Fixture id {:d} has been reset'.format(int(list_reset_fixtures[i])))

## **5. Update Group Table scores**

Below are some clever data manipulation tricks in Pandas! See if you can code it better / more efficiently.

In [14]:
# do not have to change if not desiring todo so

df_master = get_master_table(sql_teams, sql_group_fixtures, sql_group_team_results, sql_group_teams)

df_pld_indi = pd.DataFrame(df_master.groupby('fixture_id').sum()[['points']].apply(played_indicator, axis=1)).reset_index().rename(columns={0 : 'pld_indi'})
df_master_pld = df_master.merge(df_pld_indi, on='fixture_id', how='left')

df_master_pld['win_indi'] = df_master_pld.apply(win_indicator, axis=1)
df_master_pld['draw_indi'] = df_master_pld.apply(draw_indicator, axis=1)
df_master_pld['loose_indi'] = df_master_pld.apply(loose_indicator, axis=1)

team_Stats = df_master_pld.groupby('team_id').sum()[['goals_for', 'goals_against', 'pld_indi', 'points']].rename(columns={'goals_for' : 'gf', 'goals_against' : 'ga', 'pld_indi' : 'pld', 'points' : 'pts'})
wld_stats = df_master_pld.groupby('team_id').sum()[['win_indi', 'draw_indi', 'loose_indi']].rename(columns={'win_indi' : 'w','draw_indi' : 'd','loose_indi': 'l'})
team_points = df_master_pld.groupby('team_id').sum()[['points']]

df_group_teams_update = team_points.join(team_Stats).join(wld_stats).reset_index()

df_group_teams_update['group_id'] = df_group_teams_update['team_id'].map(dict_teamid_group)
df_group_teams_update = df_group_teams_update.sort_values(by=['group_id', 'team_id']).reset_index().rename(columns={'index' : 'id'})
df_group_teams_update['id'] = df_group_teams_update['id'] + 1

df_group_teams_update['gd'] = df_group_teams_update['gf'] - df_group_teams_update['ga']

_Run the code below in order to populate the Group Tables_

In [15]:
# This Code To Update
for index, row in df_group_teams_update.iterrows():
    sql_update_group_teams = """ UPDATE group_teams SET pld={:d}, w={:d}, d={:d}, l={:d}, gf={:d}, ga={:d}, gd={:d}, pts={:d} WHERE id={:d}""".format(int(row['pld']), int(row['w']), int(row['d']), int(row['l']), int(row['gf']), int(row['ga']), int(row['gd']), int(row['pts']), int(row['id']))
    sql_update_execute(sql_update_group_teams)
    print('group teams id {:d} update'.format(int(row['id'])))  

group teams id 1 update
group teams id 2 update
group teams id 3 update
group teams id 4 update
group teams id 5 update
group teams id 6 update
group teams id 7 update
group teams id 8 update
group teams id 9 update
group teams id 10 update
group teams id 11 update
group teams id 12 update
group teams id 13 update
group teams id 14 update
group teams id 15 update
group teams id 16 update
group teams id 17 update
group teams id 18 update
group teams id 19 update
group teams id 20 update
group teams id 21 update
group teams id 22 update
group teams id 23 update
group teams id 24 update
group teams id 25 update
group teams id 26 update
group teams id 27 update
group teams id 28 update
group teams id 29 update
group teams id 30 update
group teams id 31 update
group teams id 32 update
